# Global Streamflow Prediction API Demo
The Global Streamflow Prediction API provides high-resolution streamflow forecasts for regions including Africa, the Americas, and South Asia. The forecasts are derived using the Global Flood Awareness System (GloFAS) runoff and RAPID.  These results can be used for generating dynamic flood maps, predicting droughts, operating dams (to calculate how much water to release), and more.  

This demo will show how to get the historical simulation and forecast for the Rio Coco in Nicaragua/Honduras using the API.

WARNING: currently, this demo won't work while a new forecast is being built.  This process kicks off at 1:55am UTC and lasts 3 to 4 hours.  

### Imports and constants

In [ ]:
import pandas as pd
import requests
from io import StringIO
import json
from lxml import etree, html
import matplotlib.pyplot as plt
from datetime import datetime

%autosave 0

# Checkout the documentation for a list of available calls and parameters
api_base_url = 'http://global-streamflow-prediction.eastus.azurecontainer.io'

### Functions

In [ ]:
def call_api(call, params={}):
    api_request = requests.get(f'{api_base_url}/api/{call}/', params=params)
    return api_request

def format_csv_results(results):
    data = StringIO(results.text)  # Create an IO object for parsing

    df = pd.read_csv(data, index_col=0)  # Create DataFrame
    df.index = pd.to_datetime(df.index)  # Parse dates to datetime type
    data.close()

    return df

def plot_results(df):
    ax = df.plot(figsize=(12, 4), legend=False)
    ax.set_ylabel("Discharge (cms)")
    plt.show()

### Find available regions

In [ ]:
regions = call_api('AvailableRegions')
target_region = json.loads(regions.text)['available_regions'][-1]
json.loads(regions.text)

### Find available dates

In [ ]:
regions = call_api('AvailableDates', params={'region': target_region})

target_date = json.loads(regions.text)['available_dates'][0]
print(target_date)
json.loads(regions.text)

### Get forecast ensembles
There are 51 different ensemble forecasts, plus a 52nd higher-resolution forecast, that predict the streamflow in volume (cubic meters) per second.  Each ensemble presents a possible scenario by using slightly different inputs to account for uncertainty.  

The comID that is returned can be used as the reachID in subsequent calls for faster processing, as seen in the second cell below.  Each comID is associated with a specific reach and drainage area.  

The two cells below show the same data, displayed first as json and then as csv.  

In [ ]:
params = {
    'region': target_region, 
    'date': target_date, # retrieves latest available date if left out, equal "most_recent", or ""
    'ensemble': 52, # a unique ensemble number, individual ensembles separated by commas or a range separated by a "-". Retrieves all ensembles left out, equal "all", or ""
    'lat': 17.14, 
    'lon': 81.67, 
    'return_format': 'json' # csv and waterml formats also available
}

ens_results = call_api('ForecastEnsembles', params)
target_id = json.loads(ens_results.text)['comid']
json.loads(ens_results.text)

In [ ]:
params = {
    'region': target_region, 
    'reach_id': target_id,  
    'return_format': 'csv'
}

all_ens_results = call_api('ForecastEnsembles', params)
all_ens_df = format_csv_results(all_ens_results)

ensemble_values = all_ens_df.iloc[:, :-1].dropna() # get all ensembles except high resolution (# 52)
ensemble_values.head()

### Get forecast statistics
This example calculates basic statistics from ensembles (i.e. mean, min, max, and standard deviation).  It is useful to look at the entire collection of ensembles to get the most insight into streamflow predictions, but different statistics may be useful based on the scenario.  For example, when planning for drought, you might want to look at the min flow values, but an emergency response team who has to decide whether to close a road due to flooding may want to look at the max flow values.  These statistics help you make the best decisions with the predictions.  

This data is returned in [WaterML](http://www.opengeospatial.org/standards/waterml), a XML subset that is the OGC standard for sharing time-series water data.  This format is useful for many water-related tools.

In [ ]:
params = {
    'region': target_region, 
    'reach_id': target_id, # responds faster if reach_id given instead of lat lon
    'stat_type': 'mean', # min, max, and other stat types also available
    'return_format': 'waterml'
}

mean = call_api('ForecastStats', params)
print(mean.text)

### Get historic data
This "historical simulation" example predicts historical values for planning.  This provides a hindcast (prediction of what the past looked like).  In developing countries, there may not be historical data available on streamflow in the past, and this is useful if you want to build a bridge across a stream and ensure that it can span the varying water levels.  

NOTE: if you have ground-truth streamflow data available, use that!  But this is a good alternative when you don't have historical data for a location.  

In [ ]:
params = {
    'region': target_region, 
    'reach_id': target_id, 
    'return_format': 'csv'
}

historic_sim = call_api('HistoricSimulation', params)

parsed_results = format_csv_results(historic_sim)
parsed_results.head()

### Visualize the data
These two visualizations show the streamflow predictions from the various ensembles above and the historical simulation results, plotted by time.  

In [ ]:
plot_results(ensemble_values)

In [ ]:
plot_results(parsed_results)